In [2]:
import requests
from bs4 import BeautifulSoup, element
import numpy as np 
import pandas as pd 
import re

In [3]:
pages = pd.read_csv("tandf.csv")
pages = pages['pages'].tolist()
articles = pd.DataFrame(columns = ['keywords', 'title', 'authors', 'published'])

ctr = 0

for page in pages:

    prefix = re.search('https://.*com', page).group(0)
    
    page = requests.get(page)
    soup = BeautifulSoup(page.content, "lxml")
    keywords = soup.find("div", class_ = "hlFld-KeywordText")
    keywords = keywords.find_all("a")
    keywords = ', '.join([k.text.strip() for k in keywords])

    cit_ref = soup.find('li', class_ = 'downloadCitations').find('a')['href']
    cit_page = requests.get(prefix + cit_ref)
    cit_soup = BeautifulSoup(cit_page.content, 'lxml')
    citation = cit_soup.find('div', class_ = 'pub-meta').text.replace('\n', ' ').strip()

    authors = citation.split('(')[0].replace(' &', ',')
    date = re.search('(\d{4})', citation).group(0)
    title = re.search('\)[^,]*', citation).group(0)[2:]

    articles = articles.append({'authors': authors,
                                'title': title,
                                'published': date,
                                'keywords': keywords}, ignore_index=True)

display(articles)

,keywords,title,authors,published
0,"Donald Trump, sexism, voting, 2016 election, A...",Just Locker Room Talk? Explicit Sexism and the...,"Jesse H. Rhodes, Elizabeth A. Sharrow, Jill S....",2020
1,"political incivility, simplistic argumentation...",Uncivil Communication and Simplistic Argumenta...,"Ine Goovaerts, Sofie Marien",2020
2,"affective polarization, intergroup contact, in...",Can Interparty Contact Reduce Affective Polari...,"Magdalena Wojcieszak, Benjamin R. Warner",2020
3,"issue ownership, political parties, position, ...",Negatively Affecting Voters’ Issue Considerati...,"Jonas Lefevere, Henrik Bech Seeberg, Stefaan W...",2020
4,"citizens’ political talk, deliberative democra...",From the Living Room to the Meeting Hall? Citi...,"Rüdiger Schmitt-Beck, Christiane Grill",2020
...,...,...,...,...
251,"Mobile news, social media news, news effects, ...",Mobile but Not Mobilized? Differential Gains f...,Jakob Ohme,2020
252,"Journalism, social media, WhatsApp, content an...",What’s New about Whatsapp for News? A Mixed-Me...,"Karin Boczek, Lars Koppers",2020
253,"Mobile technology, mobilities, mobile journali...",Making the 21st Century Mobile Journalist: Exa...,"Matthew N. Bui, Rachel E. Moran",2020
254,"Mobile, smartphone, news, production, consumpt...",Understanding Mobile News: Looking beyond the ...,"Leopoldina Fortunati, John O’Sullivan",2020


In [12]:
pages = pd.read_csv("sage.csv")
pages = pages['pages'].tolist()
articles2 = pd.DataFrame(columns = ['keywords', 'title', 'authors', 'published'])

for page in pages:

    print(page)

    prefix = re.search('https://.*com', page).group(0)
    
    page = requests.get(page)
    soup = BeautifulSoup(page.content, "lxml")
    keywords = soup.find("div", class_ = "hlFld-KeywordText")
    keywords = keywords.find_all("a")
    keywords = ', '.join([k.text.strip() for k in keywords])

    title = soup.find('h1').text.replace('\n', ' ').strip()
    author = soup.find('div', 'hlFld-ContribAuthor').find_all('span', 'contribDegrees')
    author = ', '.join([a.find('a').text.strip() for a in author])
    date = re.search('\d{4}', soup.find('div', 'published-dates').text.split(':')[-1].strip()).group(0)

    articles2 = articles2.append({'authors': author,
                                'title': title,
                                'published': date,
                                'keywords': keywords}, ignore_index=True)

display(articles2)

https://journals.sagepub.com/doi/abs/10.1177/1464884919883489
https://journals.sagepub.com/doi/abs/10.1177/1464884919883490
https://journals.sagepub.com/doi/abs/10.1177/1464884919883491
https://journals.sagepub.com/doi/abs/10.1177/1464884919883492
https://journals.sagepub.com/doi/abs/10.1177/1464884919880060
https://journals.sagepub.com/doi/abs/10.1177/1464884917753785
https://journals.sagepub.com/doi/abs/10.1177/1464884918754851
https://journals.sagepub.com/doi/abs/10.1177/1464884918755638
https://journals.sagepub.com/doi/abs/10.1177/1464884918754829
https://journals.sagepub.com/doi/abs/10.1177/1464884918761629
https://journals.sagepub.com/doi/abs/10.1177/1464884918754849
https://journals.sagepub.com/doi/abs/10.1177/1464884918763503
https://journals.sagepub.com/doi/abs/10.1177/1464884917716503
https://journals.sagepub.com/doi/abs/10.1177/1464884918802525
https://journals.sagepub.com/doi/abs/10.1177/1464884918797613
https://journals.sagepub.com/doi/abs/10.1177/1464884917716818
https://

AttributeError: 'NoneType' object has no attribute 'find_all'

In [43]:
articles = articles.append(articles2)

In [44]:
len(articles)

268

In [45]:
articles.to_csv('keywords.csv')